In [65]:
!pip install pysolr  python-dotenv

In [2]:
import requests
import json
import pysolr

In [35]:
# Tesst endpoints
CKAN_ENDPOINT = 'https://catalog.data.gov/api/3/action/package_search'
SOLR_ENDPOINT = 'http://localhost:8984/solr/pangia'

# Define the CKAN API parameters
params = {
    'q': '*:*',
    'rows': 100,
    'start': 0,
    'sort': 'metadata_modified desc'
}

In [36]:
# Test if Solr is running
solr_client = pysolr.Solr(SOLR_ENDPOINT)
solr_client.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":4,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-11"}},\n  "status":"OK"}\n'

In [38]:
# Get the data
response = requests.get(CKAN_ENDPOINT, params=params)
data = json.loads(response.text)

{'help': 'https://catalog.data.gov/api/3/action/help_show?name=package_search', 'success': True, 'result': {'count': 250685, 'facets': {}, 'results': [{'author': None, 'author_email': None, 'creator_user_id': '2b785922-9f13-491b-a3c2-2a40acbd80c2', 'id': 'c65502e7-c4b8-419e-bd89-d8c25d64f9a0', 'isopen': False, 'license_id': 'other-license-specified', 'license_title': 'other-license-specified', 'maintainer': 'NCIRD', 'maintainer_email': 'iisinfo@cdc.gov', 'metadata_created': '2022-01-28T07:08:58.567696', 'metadata_modified': '2023-04-01T19:34:02.789553', 'name': 'covid-19-booster-dose-eligibility-in-the-united-states-cfca1', 'notes': '<p style="margin:0in;vertical-align:baseline;"><span style=\'font-size:15px;font-family:"Calibri",sans-serif;color:black;\'>This site provides historical data related to COVID-19 booster dose eligibility presented on two CDC COVID Data Tracker sites:&nbsp;</span><a href="https://covid.cdc.gov/covid-data-tracker/#vaccinations"><span style=\'font-size:15px;f

In [58]:
while True:
    for record in data['result']['results']:
        print("Adding " + record['title'] + " to Solr index.")
        solr_client.add([
            {
            'id': record['id'],
            'title': record['title'],
            'author': record['author'],
            'license': record['license_title'],
            'type': record['type'],
            'description': record['notes'],
            'resource_count': record['num_resources'],
            'record_url': 'https://catalog.data.gov/' + record['type'] + '/' + record['name']
            }
        ])

        # commit changes to Solr index
        solr_client.commit()

    # Update the CKAN API parameters to get the next page of results
    params['start'] += params['rows']
    # print(params['start'])
    # if params['start'] >= data['result']['count']:
    #     break
    break

# optimize solr index
print("Optimizing index")
solr_client.optimize()



Adding COVID-19 Booster Dose Eligibility in the United States to Solr index.
Adding COVID-19 Vaccinations in the United States,Jurisdiction to Solr index.
Adding Scorecard measure2 to Solr index.
Adding featAuto_topicArea_measureDisplayGroups to Solr index.
Adding COVID-19 Vaccination Trends in the United States,National and Jurisdictional to Solr index.
Adding COVID-19 Public Therapeutic Locator to Solr index.
Adding COVID-19 Primary Series Completion, Booster Dose Eligibility, and Booster Dose Receipt by Age, United States to Solr index.
Adding featAuto_measure_concernLevel to Solr index.
Adding Scorecard tag to Solr index.
Adding featAuto_concernLevel to Solr index.
Adding featAuto_map to Solr index.
Adding Pending Initial Logging and Tracking Non Physicians to Solr index.
Adding Order and Referring to Solr index.
Adding Provisional COVID-19 Death Counts by Week Ending Date and State to Solr index.
Adding featAuto_fileType_measureDisplayGroups to Solr index.
Adding Learning Resource

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">0</int>\n</lst>\n</response>\n'

In [64]:
# test query
results = solr_client.search("*:*")
print("\nSaw {0} result(s).".format(len(results)))

for result in results:
    print("The title is '{0}'.".format(result['title']))
    print(result)


Saw 10 result(s).
The title is '['FDIC Failed Bank List']'.
{'id': '4cfdbe83-666d-4e72-b8c6-31dbcdd8dbf0', 'title': ['FDIC Failed Bank List'], 'license': 'License not specified', 'type': ['dataset'], 'description': ['The FDIC is often appointed as receiver for failed banks. This list includes banks which have failed since October 1, 2000.'], 'resource_count': [2], 'record_url': ['https://catalog.data.gov/dataset/fdic-failed-bank-list'], '_version_': 1762076272929800192}
The title is '['Electric Vehicle Population Data']'.
{'id': '844dbad1-ee1e-44b8-9799-34cb7ed24640', 'title': ['Electric Vehicle Population Data'], 'license': 'other-license-specified', 'type': ['dataset'], 'description': ['This dataset shows the Battery Electric Vehicles (BEVs) and Plug-in Hybrid Electric Vehicles (PHEVs) that are currently registered through Washington State Department of Licensing (DOL).'], 'resource_count': [4], 'record_url': ['https://catalog.data.gov/dataset/electric-vehicle-population-data'], '_v